Zakładając, że działa to flight_id oraz mamy kolumnę w którą stronę jest lot, powiedzmy: "Direction"

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Tutaj wczytanie całego pliku jako dataFrame sparka

In [2]:
from pyspark.sql import Row
from statistics import mean
from matplotlib import pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import datediff, to_date, lit
import json

filepath = 'flights_reduces.text'

sc = spark.sparkContext

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
filtered = flightsRDD.filter(lambda line: line != header)
parts = filtered.map(lambda line: line.split(';'))
flights = parts.map(lambda p:
                   Row(Scrap_date = p[0],
                       Scrap_time = p[1],
                       Country_from = p[2],
                       Country_to = p[3],
                       Flight_id = int(p[4]),
                       Flight_date = p[5],
                       Airline = p[6],
                       Change = int(p[7]),
                       Price = float(p[8]),
                       Depart_hour = p[9],
                       Depart_from = p[10],
                       Arrival_hour = p[11],
                       Arrive_to = p[12]
                      ))

Dodajemy zmienną days do każdego rekordu, jaka jest różnica pomiędzy datą wylotu, a datą sprawdzenia ceny

In [4]:
dt = spark.createDataFrame(flights)
dt = dt.withColumn("Days", datediff(to_date(dt.Flight_date, 'dd/MM/yy'), to_date(dt.Scrap_date, 'yyyy-MM-dd')))
dt = dt.withColumn("Direction", lit('There'))
dt.createOrReplaceTempView("flights")
dt.printSchema()
#dt.select('Direction').show(10)

root
 |-- Airline: string (nullable = true)
 |-- Arrival_hour: string (nullable = true)
 |-- Arrive_to: string (nullable = true)
 |-- Change: long (nullable = true)
 |-- Country_from: string (nullable = true)
 |-- Country_to: string (nullable = true)
 |-- Depart_from: string (nullable = true)
 |-- Depart_hour: string (nullable = true)
 |-- Flight_date: string (nullable = true)
 |-- Flight_id: long (nullable = true)
 |-- Price: double (nullable = true)
 |-- Scrap_date: string (nullable = true)
 |-- Scrap_time: string (nullable = true)
 |-- Days: integer (nullable = true)
 |-- Direction: string (nullable = false)



Możemy robić to w ten sposób np. chcemy lot w jedną stronę. No to wybieramy wszystkie loty ze znacznikiem "There" i grupujemy cenę po id_lotu.

In [11]:
direction = 'There'
country_from = 'Austria'
country_to = 'Bulgaria'
scrap_time = '05:00'

query = "SELECT * FROM flights WHERE Country_from = '{}' AND Country_to = '{}' \
AND Scrap_time = '{}' AND Direction = '{}'".format(country_from, country_to, scrap_time, direction)

processed = spark.sql(query)
processed.agg(F.sum(processed.Price).alias('Price_sum')).show()

+-----------------+
|        Price_sum|
+-----------------+
|4367.999999999997|
+-----------------+



In [3]:
df = spark.read.csv("raw.csv", sep = ';', header=True, inferSchema=True, mode = 'DROPMALFORMED') #to inferSchema to tak roboczo
df.printSchema()

root
 |-- Scrap_date: timestamp (nullable = true)
 |-- Scrap_time: string (nullable = true)
 |-- Country_from: string (nullable = true)
 |-- Country_to: string (nullable = true)
 |-- Flight_id: integer (nullable = true)
 |-- Flight_date: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Change: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Depart_hour: string (nullable = true)
 |-- Depart_from: string (nullable = true)
 |-- Arrival_hour: string (nullable = true)
 |-- Arrive_to: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Index: integer (nullable = true)



In [ ]:
import pyspark.sql.functions as F

df2 = df.groupBy(["Flight_id"]).agg(F.first(df.Scrap_date).alias('Scrap_date'), F.first(df.Scrap_time).alias('Scrap_time'), 
                             F.sum(df.Price).alias("Price"))
df2.groupBy(["Scrap_date", "Scrap_time"]).agg(F.min(df2.Price).alias('Price')).show()